In [1]:
from helper_functions import xor_hex_strings, single_byte_xor, repeating_key_xor, xor_bytes, hamming_distance
from file_loading import load_file_as_bytes, load_file_as_b64
from conversions import int2bytes, hex2bytes, string2bytes, hex2b64
from symmetric_encryption import is_ecb_mode, aes128_ecb_encrypt, aes128_ecb_decrypt, aes128_cbc_encrypt, aes128_cbc_decrypt, encryption_oracle
from padding import apply_pkcs_7, remove_pkcs_7
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 9: Implement PKCS#7 padding**

In [2]:
apply_pkcs_7(b"YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## **Challenge 10: Implement CBC mode**

In [3]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)
text = b'YELLOW SUBMARINE YELLOW SUBMARIN'

aes128_cbc_decrypt(aes128_cbc_encrypt(text, key, iv), key, iv)

b'YELLOW SUBMARINE YELLOW SUBMARIN'

In [4]:
file_bytes = load_file_as_b64("challenge_data/10.txt", remove_newlines=True)

key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
aes128_cbc_decrypt(file_bytes, key, IV).decode('utf-8')

"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take a

## **Challenge 11: An ECB/CBC detection oracle**

In [5]:
input_bytes = b'A' * 3 * 16
for _ in range(1000):
    oracle_mode, cyphertext = encryption_oracle(input_bytes)
    ecb_mode = is_ecb_mode(cyphertext)

    if ecb_mode and oracle_mode == "CBC":
        print("ERROR: Oracle detection is incorrect, ECB mode detected instead of CBC")
    if not ecb_mode and oracle_mode == "ECB":
        print("ERROR: Oracle detection is incorrect, CBC mode detected instead of ECB")

print("if no error is printed, the oracle is working correctly!")

if no error is printed, the oracle is working correctly!
